In [3]:
import pandas as pd
import datetime as dt
import time
import re
import warnings
import logging

warnings.filterwarnings('ignore')

from functools import reduce

from sklearn import metrics

from packages import v2models


list index out of range


In [ ]:
#DAM Price data

def read_price_data(filename='Datasets/DAM_Prices_2021-Sep23.csv', columns=['DeliveryPeriod', 'EURPrices'], date_format='%m/%d/%Y %H:%M'):
    """
    Reads in day-ahead market prices from a single CSV file.

    Parameters
    ----------
        filename : str
            Filename containing day-ahead market price data.
        columns : list of str
            List of columns to read from the file.
        date_format : str, default='%m/%d/%Y %H:%M'
            Date format to be parsed from string to datetime for the date columns in the dataset.

    Returns
    -------
        price_data : pandas.DataFrame
            Pre-processed DAM price data.
    """
    # Print the columns available in the file for debugging
    temp_df = pd.read_csv(filename, nrows=5)
    print(f"Available columns in the CSV file: {temp_df.columns.tolist()}")

    # Ensure the columns match those in the CSV
    if not all(col in temp_df.columns for col in columns):
        raise ValueError(f"Specified columns {columns} do not match the columns in the CSV file.")

    # Create date parser
    date_parse = lambda date: dt.datetime.strptime(date, date_format) + dt.timedelta(hours=1)
    
    # Read in dataset
    price_data = pd.read_csv(filename, usecols=columns, parse_dates=['DeliveryPeriod'], date_parser=date_parse)
    price_data.set_index('DeliveryPeriod', inplace=True)
    
    # Do DST adjustment - 23-hour days have their missing hour replaced with the average of the two days before and after the missing hour.
    #                     25-hour days have the two same hours replaced by their average.
    price_data = price_dst_adjustment(price_data)

    return price_data

def find_dst_index(time_step_id_dataframe, number_of_hours):
    """
    Given a set of datetime.hour values for a given date (freq='H'), return the index/hour that is either missing (if
    number_of_hours==23) or appears twice (if number_of_hours==25).

    Parameters
    ----------
        time_step_id_dataframe : pandas.core.indexes.numeric.Int64Index
        number_of_hours : int

    Returns
    -------
        time_step_id : int 
            The missing or duplicated hour for the given DST day.
    """
    if number_of_hours == 23:
        for i, time_step_id in enumerate(time_step_id_dataframe):
            if i < time_step_id:
                return i + 1
            elif i == number_of_hours - 1:
                return 23
                
    elif number_of_hours == 25:
        for j, time_step_id in enumerate(time_step_id_dataframe):
            if j + 1 > time_step_id:
                return time_step_id

def price_dst_adjustment(df):
    """ 
    Given a dataframe (of electricity prices), make DST adjustments so that days with 23 or 25 hours
    are imputed/reduced to 24 hours using simple averaging.

    Parameters
    ----------
        df : pandas.DataFrame
            Unadjusted electricity prices dataframe.

    Returns
    -------
        df : pandas.DataFrame
            Adjusted electricity prices dataframe.
    """
    df_count = df.groupby([df.index.date]).count()
    dst_dates = df_count.loc[(df_count['EURPrices']) != 24, :]

    if dst_dates.shape[0] == 0:
        return df

    for i in range(dst_dates.shape[0]):
        dst_date = dst_dates.index[i]
        number_of_hours = dst_dates.iloc[i, 0]
        df_dst_data = df.loc[df.index.date == dst_date]
        dst_index = find_dst_index(df_dst_data.index.hour, number_of_hours)
        
        if number_of_hours == 23:
            previous_price = df_dst_data.loc[df_dst_data.index.hour == dst_index - 1]
            next_day_price = df.loc[df.index.date == dst_date + dt.timedelta(days=1)]
            next_price = next_day_price.loc[next_day_price.index.hour == 0]
            adjacent_prices = pd.concat([previous_price, next_price])
            average_values = adjacent_prices.mean(axis=0).values[0]
            new_price = pd.DataFrame(dict(EURPrices=average_values), index=[dt.datetime.combine(dst_date, dt.datetime.min.time()) + dt.timedelta(hours=dst_index)])
            df = pd.concat([df, new_price])
            df.index = pd.to_datetime(df.index)
            df.index.name = 'DeliveryPeriod'
            
        elif number_of_hours == 25:
            duplicate_prices = df_dst_data.loc[df_dst_data.index.hour == dst_index]
            average_values = duplicate_prices.mean(axis=0).values[0]
            df.drop(duplicate_prices.index, inplace=True)
            new_price = pd.DataFrame(dict(EURPrices=average_values), index=[dt.datetime.combine(dst_date, dt.datetime.min.time()) + dt.timedelta(hours=dst_index)])
            df = pd.concat([df, new_price])
            df.index = pd.to_datetime(df.index)

    df.sort_index(axis=0, inplace=True)
    df = df.loc[~df.index.duplicated()]

    return df



In [11]:
#IDA price data

def read_intraday_price_data(filenames=['Datasets/IDA1_Prices_2021-Sep23.csv', 'Datasets/IDA2_Prices_2021-Sep23.csv', 'Datasets/IDA3_Prices_2021-Sep23.csv'], 
                             columns=['timestamp', 'price_eur'], 
                             date_format='%m/%d/%Y %H:%M'):
    """
    Reads in the intraday auction prices from Datasets directory.

    Parameters
    ----------
        filenames : list of str
            List of filenames containing intraday auction price data.
        columns : list of str
            List of columns to read from the files.
        date_format : str, default='%m/%d/%Y %H:%M'
            Date format to be parsed from string to datetime for the date columns in the dataset.

    Returns
    -------
        ida_data : pandas.DataFrame
            Pre-processed intraday auction price data.
    """
    # Create date parser
    date_parse = lambda date: dt.datetime.strptime(date, date_format)

    # Read in datasets and rename the price column to differentiate between IDA1, IDA2, and IDA3
    ida_data_frames = []
    for i, filename in enumerate(filenames):
        ida_data = pd.read_csv(filename, usecols=columns, parse_dates=True, index_col='timestamp', date_parser=date_parse)
        ida_data.columns = [f'IDA{i+1}_Price']
        ida_data_frames.append(ida_data)

    # Combine datasets
    ida_data = reduce(lambda left, right: left.join(right, how='outer'), ida_data_frames)

    # Remove duplicates
    ida_data = ida_data.loc[~ida_data.index.duplicated()]

    # Remove last day of data if last delivery hour is not 23:00
    if ida_data.index[-1].hour != 21:
        last_date = dt.datetime.combine(ida_data.index[-1].date(), dt.datetime.min.time()) - dt.timedelta(hours=1)
        ida_data = ida_data.loc[:last_date]

    # Get list of dates that do not have exactly 24 data points (hours)
    count_df = ida_data.groupby(ida_data.index.date).count()
    missing_values = count_df.loc[(count_df != 24).any(axis=1)]

    # Clean data - Use the backfill and forward fill method to deal with missing values
    
    ida_data = ida_data.fillna(method='ffill').fillna(method='bfill')
    
    # Relabel the index
    ida_data.index.name = 'DeliveryPeriod'

    return(ida_data)




In [ ]:
#Fuel Mix data

def read_fuel_mix_data(filename='Datasets/Fuel_Mix.csv', 
                             columns=['Period', 'Battery','CCGT','CHP','DSR','Hydro','Interconnectors','OCGT','Oil','Peat','Pumped_Storage','Solar_Actual','Waste','Wind_Actual'], 
                             date_format='%d/%m/%Y %H:%M'):
    """
    Reads in fuel mix from Datasets directory.

    Parameters
    ----------
        filename : list of str
            List of filenames containing fuel mix data.
        columns : list of str
            List of columns to read from the file.
        date_format : str, default='%d/%m/%Y %H:%M'
            Date format to be parsed from string to datetime for the date columns in the dataset.

    Returns
    -------
        fuel_mix_data : pandas.DataFrame
            Pre-processed fuel mix data provided by EnAppSys.
    """
    # Create date parser
    date_parse = lambda date: dt.datetime.strptime(date, date_format)

    # Read in dataset
    fuel_mix_data = pd.read_csv(filename, usecols=columns, parse_dates=True, index_col='Period', date_parser=date_parse)
    
    # Remove last day of data if last delivery hour is not 23:00
    if fuel_mix_data.index[-1].hour != 21:
        last_date = dt.datetime.combine(fuel_mix_data.index[-1].date(), dt.datetime.min.time()) - dt.timedelta(hours=1)
        fuel_mix_data = fuel_mix_data.loc[:last_date]

    # Get list of dates that do not have exactly 24 data points (hours)
    count_df = fuel_mix_data.groupby(fuel_mix_data.index.date).count()
    missing_values = count_df.loc[(count_df != 24).any(axis=1)]
    
    # Relabel the index
    fuel_mix_data.index.name = 'DeliveryPeriod'

    return(fuel_mix_data)



In [ ]:
def read_gas_price_data(filename='Datasets/TTF_gas_prices.csv', 
                             columns=['Period', 'GasPrice'], 
                             date_format='%m/%d/%Y %H:%M'):
    
    """
    Reads in gas price from Datasets directory. TTF gas futures used as proxy for gas price.

    Parameters
    ----------
        filename : list of str
            List of filenames containing gas price data.
        columns : list of str
            List of columns to read from the files.
        date_format : str, default='%m/%d/%Y %H:%M'
            Date format to be parsed from string to datetime for the date columns in the dataset.

    Returns
    -------
        gas_price_data : pandas.DataFrame
            Pre-processed gas price data.
    """
    # Create date parser
    date_parse = lambda date: dt.datetime.strptime(date, date_format)
    
    #Read in Dataset
    gas_price_data = pd.read_csv(filename, usecols=columns, parse_dates=True, index_col='Period', date_parser=date_parse)
    
    #Set Index name
    gas_price_data.index.name = 'DeliveryPeriod'

    return(gas_price_data)
    

In [2]:
# Demand/Wind/Solar forecast data

def read_forecast_data(forecast_type, filename=None, forecast_column=['Period', 'AggregatedForecast'], date_format='%m/%d/%Y %H:%M'):
    """
    Reads in forecast demand, solar or wind load from Datasets directory.
    Parameters
    ----------
        forecast_type : string in ['Wind', 'Forecast', 'Solar']
        filename : list of str
            List of filenames containing the forecast data.
        forecast_column : list of str
            List of columns to read from the files.
        date_format : str, default='%Y-%m-%d %H:%M:%S'
            Date format to be parsed from string to datetime for the date columns in the dataset.
    Returns
    -------
        forecast_data : pandas.DataFrame
            Pre-processed forecast data provided by EnAppSys.
    """
    # Set the filenames to be read in when forecast_type is specified.
    if filename is None:
        if forecast_type == 'Wind':
            filename = ['Datasets/Onshore_Wind_Forecast1Sep22-Oct23.csv', 'Datasets/Onshore_Wind_Forecast2Sep22-Oct23.csv']
        elif forecast_type == 'Demand':
            filename = ['Datasets/Demand_Forecast1Sep22-Oct23.csv', 'Datasets/Demand_Forecast2Sep22-Oct23.csv']
        elif forecast_type == 'Solar':
            filename = ['Datasets/Solar_Forecast1Sep22-Oct23.csv', 'Datasets/Solar_Forecast2Sep22-Oct23.csv']

    # Create date parser
    date_parse = lambda date: dt.datetime.strptime(date, date_format) + dt.timedelta(hours=1)

    # Read in dataset/s
    forecast_data1 = pd.read_csv(filename[0], usecols=forecast_column, parse_dates=True, index_col='Period', date_parser=date_parse)
    forecast_data2 = pd.read_csv(filename[1], usecols=forecast_column, parse_dates=True, index_col='Period', date_parser=date_parse)

    # Combine dataset/s
    forecast_data = pd.concat([forecast_data1, forecast_data2])

    # Remove duplicates
    forecast_data = forecast_data.loc[[not val for val in forecast_data.index.duplicated()]]

    # Rename index and column
    forecast_data.index.name = 'DeliveryPeriod'
    forecast_data.columns = [forecast_type]

    return(forecast_data)


In [8]:
# Walk-forward evaluation of a forecasting model

def walk_forward_evaluation(model, price_data, ida_data=None, planned_data=None, fuel_mix_data=None, gas_price_data=None, starting_window_size=None, moving_window=False, start=None, end=None, logs=True):
    """
    Evalutes a forecasting model using the given price_data and (optional) ida market data,
    wind/solar/demand forecast data, fuel mix data and gas price data. The general procedure is as follows:
        
        * Create an initial training data window. It is important to that only the data that would be available the day before
          the first forecast date given by the start parameter is used.
        * For each forecast_date in dates_between(start, end):
            - Ingest data. The data required by the model is reformatted (as needed) into suitable train/test input
              and stored in the model object for later training/forecasting.
            - Train. The model is trained to the available data prior to forecast_date.
            - Forecast. Forecast is generated for forecast_date.
            - Store forecasts in a dataframe.
        * Calculate RMSE and MAE for the whole period.
        * Return model object, model forecasts, RMSE and MAE.

    The above procedure applies for the following model classes: naive(), random_forest(), ARX(), SARIMAX(), ffnn(), rnn().

    Parameters
    ----------
        model : class
            Class object defining the forecasting model, with class methods
            self.ingest_data(), self.train() and self.forecast().
        price_data : pandas.DataFrame
            Electricity price data.
        ida_data : pandas.DataFrame
            Intraday Auction market price data.
        planned_data : pandas.DataFrame
            Wind, Solar and demand forecast data.
        fuel_mix_data : pandas.DataFrame
            DAM supply and demand curve data.
        gas_price_data: pandas.DataFrame
            TTF futures gas price data used as proxy for gas data.
        starting_window_size : int
            number of days of data to start the training set on, i.e. training set size
            for first forecast iteration on start date.
        moving_window : bool
            To specify whether the training window is a moving window (True) or an expanding window (False).
        start : datetime.datetime
            Date on which to start the walk-forward validation.
        end : datetime.datetime
            Date to end the walk-forward validation on (inclusive).
        logs : bool, default=True
            Specifies whether to print overall execution time of the walk-forward validation
            for the entire start-to-end period. True prints out the logs.

    Returns
    -------
        model : class
            The model object after it has been modified by the walk_forward_evaluation() function.
        forecasts_df : pandas.DataFrame
            DataFrame of original prices and corresponding model forecasts
        rmse : int
            Root-mean-square error (RMSE) for the entire period represented by forecasts_df
        mae : int
            Mean absolute error (MAE) for the entire period represented by forecasts_df
"""
    if logs:
        start_time = time.time()
    
    # Validation to ensure start date <= end date.
    if start is not None and end is not None:
        if start > end:
            raise Exception(f"Cannot have start date after end date.")
    
    # Validation to ensure the number of days of training data is at least starting_window_size.
    if starting_window_size is not None:
        if (start-price_data.index[0]).days < starting_window_size:
            raise Exception(f"Not enough data for training: starting_window_size={starting_window_size}, train_size={start-data.index[0]}")
    
    # Fetch datetime index for initial training data window
    if starting_window_size is None:
        train_dates = list(pd.date_range(start=price_data.index[0], end=start-dt.timedelta(hours=1), freq='H'))
    else:
        train_dates = list(pd.date_range(end=start-dt.timedelta(hours=1), periods=24*starting_window_size, freq='H'))

    # Get initial price data
    train_price_data = price_data.loc[train_dates,:]
    
    # Get initial IDA market data
    train_ida_data = None if ida_data is None else ida_data.loc[train_dates,:]
    
    # Fetch datetime index for initial planned (wind, solar & forecast) data
    if planned_data is not None:
        if starting_window_size is None:
            train_planned_dates = pd.date_range(start=planned_data.index[0], end=start+dt.timedelta(hours=23), freq='H')
        else:
            train_planned_dates = pd.date_range(end=start+dt.timedelta(hours=23), periods=24*(starting_window_size+1), freq='H')
    
    # Get initial planned (wind, solar & demand) data
    train_planned_data = None if planned_data is None else planned_data.loc[train_dates,:]
    
    # Fetch datetime index for initial planned (wind, solar & forecast) data
    if fuel_mix_data is not None:
        if starting_window_size is None:
            train_fuel_mix_dates = pd.date_range(start=planned_data.index[0], end=start+dt.timedelta(hours=23), freq='H')
        else:
            train_fuel_mix_dates = pd.date_range(end=start+dt.timedelta(hours=23), periods=24*(starting_window_size+1), freq='H')
    
    # Get initial fuel mix data
    train_fuel_mix_data = None if fuel_mix_data is None else fuel_mix_data.loc[train_dates,:]
    
#     # Fetch datetime index for initial gas price data
    if gas_price_data is not None:
        if starting_window_size is None:
            train_gas_price_dates = pd.date_range(start=planned_data.index[0], end=start+dt.timedelta(hours=23), freq='H')
        else:
            train__dates_gas_price= pd.date_range(end=start+dt.timedelta(hours=23), periods=24*(starting_window_size+1), freq='H')
    
    # Get initial gas price data
    train_gas_price_data = gas_price_data.loc[train_dates,:]
        
    # Create dataframe to store errors
    forecast_index = pd.date_range(start=start, end=end+dt.timedelta(hours=23), freq='H')
    forecasts_df = pd.DataFrame(columns=['Forecast'], index=forecast_index)
    forecasts_df.insert(0, 'Original', price_data['EURPrices'].loc[forecast_index])
    
    # Debugging print for initial setup
    print("Initial setup complete.")
    print(f"Train Price Data Index: {train_price_data.index}")
    if ida_data is not None:
        print(f"Train IDA Data Index: {train_ida_data.index}")
    if planned_data is not None:
        print(f"Train Planned Data Index: {train_planned_data.index}")
    if fuel_mix_data is not None:
        print(f"Train Fuel Mix Data Index: {train_fuel_mix_data.index}")
    if gas_price_data is not None:
        print(f"Train Gas Price Data Index: {train_gas_price_data.index}")
    
    # Loop through data to train and forecast iteratively over the expanding (or moving) window
    for i in range((end-start).days+1):
        current_time = start + dt.timedelta(days=i)
        next_day = current_time + dt.timedelta(days=1)
        
        # Ingest data
        if fuel_mix_data is not None:
            model.ingest_data(train_price_data, train_ida_data, train_planned_data, train_fuel_mix_data, train_gas_price_data)
        else:
            model.ingest_data(train_price_data, train_ida_data, train_planned_data, None)
        
        # Train model
        model.train()
        
        # Generate day-ahead forecast and store in forecasts_df dataframe
        forecast = model.forecast()
        
        # Adjust forecast index to match the expected date range
        forecast_index = pd.date_range(start=current_time, end=next_day-dt.timedelta(hours=1), freq='H')
#         forecast = pd.DataFrame(forecast, index=forecast_index)
        
        # Debugging prints before updating forecasts_df
        print(f"Current iteration: {i}")
        print(f"Forecast index: {forecast.index if hasattr(forecast, 'index') else 'N/A'}")
        print(f"Forecast values: {forecast.values}")
        
        try:
            forecasts_df.loc[forecast.index, 'Forecast'] = forecast.values
        except KeyError as e:
            print(f"KeyError encountered: {e}")
            print(f"Current Time: {start + dt.timedelta(days=i)}")
            print(f"Forecasts DataFrame Index: {forecasts_df.index}")
            print(f"Forecast index: {forecast.index}")
            print(f"Forecast values: {forecast.values}")
            raise
        forecast_value = forecast.iloc[0, 0]
        
        # Drop last day of data if moving_window=True
        if moving_window:
            train_price_data.drop(train_price_data.index[:24], inplace=True)
            if ida_data is not None:
                train_ida_data.drop(train_ida_data.index[:24], inplace=True)
            if planned_data is not None:
                train_planned_data.drop(train_planned_data.index[:24], inplace=True)
            if fuel_mix_data is not None:
                train_fuel_mix_data.drop(train_planned_data.index[:24], inplace=True)
            if gas_price_data is not None:
                train_gas_price_data.drop(train_planned_data.index[:24], inplace=True)
        
        # Get datetime index for new date of data to be added to training data
        next_date = list(pd.date_range(start=train_price_data.index[-1]+dt.timedelta(hours=1), periods=24, freq='H'))
        
        # Fetch new DAM prices data and add to training data
        new_price_data = price_data.loc[next_date,:]
        train_price_data = pd.concat([train_price_data, new_price_data])
        
        print(f"Finished forecast for {forecast.index.date[0]}. Forecast value: {forecast_value}.")
        
        # Fetch new IDA prices data and add to training data
        if ida_data is not None:
            new_ida_data = ida_data.loc[next_date,:]
            train_ida_data = pd.concat([train_ida_data, new_ida_data])
            
        # Fetch new forecast (wind & demand) data and add to training data
        if planned_data is not None:
            #next_planned_date = list(pd.date_range(start=train_planned_data.index[-1]+dt.timedelta(hours=1), periods=24, freq='H'))
            new_planned_data = planned_data.loc[next_date,:]
            train_planned_data = pd.concat([train_planned_data, new_planned_data])
            
        # Fetch new fuel mix data and add to training data
        if fuel_mix_data is not None:
            #next_fuel_mix_date = list(pd.date_range(start=train_fuel_mix_data.index[-1]+dt.timedelta(hours=1), periods=24, freq='H'))
            new_fuel_mix_data = fuel_mix_data.loc[next_date,:]
            train_fuel_mix_data = pd.concat([train_fuel_mix_data, new_fuel_mix_data])
        
        # Fetch new gas price data and add to training data
        if gas_price_data is not None:
            #next_gas_price_date = list(pd.date_range(start=train_gas_price_data.index[-1]+dt.timedelta(hours=1), periods=24, freq='H'))
            new_gas_price_data = gas_price_data.loc[next_date,:]
            train_gas_price_data = pd.concat([train_gas_price_data, new_gas_price_data])
            
    # Calculate RMSE and MAE for the entire period of the walk-forward evaluation
    rmse = metrics.mean_squared_error(forecasts_df['Original'], forecasts_df['Forecast'], squared=False)
    mae = metrics.mean_absolute_error(forecasts_df['Original'], forecasts_df['Forecast'])
    
    if logs:
        print(f"Execution time: {time.time()-start_time} seconds")
        
    return(model, forecasts_df, rmse, mae)


def get_resampled_errors(res_df, index_filter):
    """
    This takes a dataframe of forecasted (and original) hourly electricity price values
    and calculates rmse/mae values across different sampling rates.

    Parameters
    ----------
        res_df : pandas.DataFrame
            DataFrame of original and forecast prices.
        index_filter : string in ['date', 'dayofweek_and_hour', "'ayofweek', 'hour', 'month']
            Sampling rate.

    Returns
    -------
        errors : pandas.DataFrame
    """
    # Group dataframe values by index, with grouping determined by index_filter
    if index_filter == 'date':
        group = res_df.groupby(res_df.index.date)
    elif index_filter == 'dayofweek_and_hour':
        group = res_df.groupby([res_df.index.hour, res_df.index.dayofweek])
    elif index_filter == 'dayofweek':
        group = res_df.groupby(res_df.index.dayofweek)
    elif index_filter == 'hour':
        group = res_df.groupby(res_df.index.hour)
    elif index_filter == 'month':
        group = res_df.groupby(res_df.index.month)
        
    # Calculate RMSEs and MAEs for each group period
    rmses = group.apply(lambda df: metrics.mean_squared_error(df['Original'], df['Forecast'], squared=False))
    maes = group.apply(lambda df: metrics.mean_absolute_error(df['Original'], df['Forecast']))
    
    # Combine RMSEs and MAEs dataframes into one dataframe
    errors = pd.concat([rmses, maes], axis=1)
    errors.columns = ['RMSE', 'MAE']
    
    return(errors)


def walk_forward_loop(model_func, dates_to_forecast, model_params, lag_params, hyperparameter, prices, ida_prices, planned, fuel_mix, gas_price, logs=True):
    """
    A looping function to use the walk-forward validation on a set of non-adjacent (datetime) dates.
    This function is used for Section 4.4 - Hyperparameter Tuning.

    Parameters
    ----------
        model_func : class
            Class object defining the forecasting model, with class methods
            self.ingest_data(), self.train() and self.forecast().
        dates_to_forecast : list of datetime.datetime
            Dates to forecast on.
        model_params : dict
            Argument for model __init__() specifying model parameters
        lag_params : dict
            Argument for model __init__() specifying the data lags to be used as (exogenous) predictors
        hyperparameter : str
            Name of model hyperparameter to iteratively parameterise the model on.
            Note: the corresponding hyperparameter in the model_params must be a list of possible hyperparameter
            values to train the models on.
        prices : pandas.DataFrame
            Electricity price data.
       IDA_prices : pandas.DataFrame
            Intraday Auction market price data.
        planned : pandas.DataFrame
            Wind and demand forecast data.
        fuel_mix : pandas.DataFrame
            Fuel mix data
        gas_price_data: pandas.DataFrame
            TTF futures gas price data used as proxy for gas data.
        logs : bool, default=True
            Specifies whether to print overall execution time of the walk-forward validation
            for the entire start-to-end period. True prints out the logs.

    Returns
    -------
        errors : pandas.DataFrame
            Dataframe of overall RMSE and MAE of each hyperparameter value.
    """
    # Initialise dataframe to store errors (indexed by hyperparameter values given by model_params["init_params"][hyperparameter]).
    errors_index = model_params['init_params'][hyperparameter] if model_func == v2models.ffnn else model_params[hyperparameter]
    errors = pd.DataFrame(columns=['RMSE', 'MAE'], index=errors_index)
    errors.index.name = hyperparameter
    
    # Ensure that dates_to_forecast values are of type datetime.datetime
    dates_to_forecast = [dt.datetime.combine(date, dt.datetime.min.time()) if type(date) is dt.date else date for date in dates_to_forecast]

    # Fix hyperparameter value for walk-forward validation
    for param in errors.index:
        new_model_params = model_params.copy()
        
        # Create appropriate model_params dict as input for model __init__().
        if model_func == v2models.ffnn:
            new_model_params['init_params'][hyperparameter] = param
        else:
            new_model_params[hyperparameter] = param

        overall_res = pd.DataFrame(columns=['Original', 'Forecast'])
        
        # Run walk-forward validation for all dates in dates_to_forecast.
        for date in dates_to_forecast:
            # Initialise model
            model = model_func(model_params=new_model_params, lag_params=lag_params)
            
            # Run walk-forward validation function for the given date in dates_to_forecast,
            # and temporarily store the forecasts
            _, res, _, _ = walk_forward_evaluation(model, prices, ida_prices, planned, fuel_mix, gas_price, start=date, end=date, logs=False)
            overall_res = overall_res.append(res)
            
        # Calculate and store the RMSE and MAE for the forecasts over all the dates in dates_to_forecast
        rmse = metrics.mean_squared_error(overall_res['Original'], overall_res['Forecast'], squared=False)
        mae = metrics.mean_absolute_error(overall_res['Original'], overall_res['Forecast'])
            
        # Store the RMSE and MAE for the given hyperparameter.
        errors.loc[param, 'RMSE'] = rmse
        errors.loc[param, 'MAE'] = mae
        
        if logs: print(f"Finished for {hyperparameter}={param}")
    
    return(errors)